In [1]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, curdoc
from bokeh.themes import Theme
from bokeh.models import HoverTool
from bokeh.palettes import Spectral11
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.charts import Histogram, Donut
from bokeh.io import output_notebook
from bokeh.models.formatters import DatetimeTickFormatter as dttf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [2]:
df_clean = pd.read_json('messy.json')
df_clean

,Age_at_win,AverageWinsToDate,CountryCode,CumulativePrizeToDate,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,Finals,...,country,currency,date,num_teams,rankplace_Cash Only,rankplace_Runners Up,rankplace_Winners,team_history,teams,totalsc2
0,23.0,0.000000,ua,136,EUR,DIMAGA,5.181408e+11,1267315200000,1.361800,0,...,ua,EUR,2010-02-28,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Unaffiliated,85
1,23.0,136.000000,ua,273,EUR,DIMAGA,5.181408e+11,1268524800000,1.376900,1,...,ua,EUR,2010-03-14,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Unaffiliated,85
10,24.0,638.400000,ua,6651,EUR,DIMAGA,5.181408e+11,1295740800000,1.336525,9,...,ua,EUR,2011-01-23,5,0,0,1,"[{'team': 'IsIMBA', 'history': [1461024000000,...",Other,85
100,20.0,11777.400000,kr,201661,USD,MC,6.771168e+11,1310256000000,1.000000,9,...,kr,USD,2011-07-10,5,0,1,0,"[{'team': 'CJ Entus', 'history': [146396160000...",Old Generations,108
1000,17.0,816.250000,kr,11888,KRW,RagnaroK,8.417088e+11,1385164800000,0.000942,4,...,kr,KRW,2013-11-23,4,0,1,0,"[{'team': 'CJ Entus', 'history': [142188480000...",Other,38
10000,NaN,0.000000,cn,244,USD,Blue,NaN,1452384000000,1.000000,0,...,cn,USD,2016-01-10,0,0,1,0,[],Unaffiliated,1
10001,NaN,0.000000,no,182,NOK,Adrian,NaN,1428192000000,0.121850,0,...,Other,Other,2015-04-05,0,0,0,1,None,Unaffiliated,2
10002,NaN,182.000000,no,240,NOK,Adrian,NaN,1458950400000,0.117728,1,...,Other,Other,2016-03-26,0,1,0,0,None,Unaffiliated,2
10003,NaN,0.000000,ca,150,USD,Imagine,NaN,1342310400000,1.000000,0,...,Other,USD,2012-07-15,1,1,0,0,"[{'team': 'FlipSid3 Tactics', 'history': [1404...",Unaffiliated,3
10004,NaN,150.000000,ca,226,USD,Imagine,NaN,1430611200000,1.000000,0,...,Other,USD,2015-05-03,1,1,0,0,"[{'team': 'FlipSid3 Tactics', 'history': [1404...",Other,3


In [39]:
to_drop = [ 'rankplace_Cash Only', 'rankplace_Runners Up', 'rankplace_Winners',
            'CountryCode','CurrencyCode','CurrentHandle','ExchangeRate','GameId','NameFirst','NameLast','Note',
            'PlayerId','Prize','RankText','TeamPlayers','Team_On','TournamentName','team_history',
            'Team_On_Backup','DOB','EndDate','TotalUSDPrize', 'Prize_USD',
            'num_teams','date','totalsc2', 'CumulativePrizeToDate',
            'TournLength','Finals', 'WinsToDate','TTtoDate']
test_here = ['SecondWins','AverageWinsToDate','PriorWins']
df_dropped = df_clean.drop(to_drop, axis=1)

In [40]:
df_dropped.head()

,Age_at_win,AverageWinsToDate,PriorWins,PrizeToDate,Rank,SecondWins,country,currency,teams
0,23.0,0.00,0,0,Winners,0,ua,EUR,Unaffiliated
1,23.0,136.00,1,136,Winners,0,ua,EUR,Unaffiliated
10,24.0,638.40,8,6384,Winners,1,ua,EUR,Other
100,20.0,11777.40,6,176661,Runners Up,3,kr,USD,Old Generations
1000,17.0,816.25,3,9795,Runners Up,1,kr,KRW,Other


In [41]:
#prep for classification
df_dums = pd.get_dummies(data=df_dropped, columns = ['Rank','currency','teams','country'])
df_dums['Age_at_win'] = df_dums['Age_at_win'].fillna(df_dums['Age_at_win'].mean())
df_dums['y']= df_dums['Rank_Runners Up'] + df_dums['Rank_Winners']
df_class = df_dums.drop(['Rank_Runners Up','Rank_Cash Only','Rank_Winners'], axis=1)
y_out = df_class.pop('y')
df_class.head()

,Age_at_win,AverageWinsToDate,PriorWins,PrizeToDate,SecondWins,currency_AUD,currency_CNY,currency_EUR,currency_KRW,currency_Other,...,country_Other,country_cn,country_de,country_fr,country_kr,country_pl,country_ru,country_se,country_ua,country_us
0,23.0,0.00,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,23.0,136.00,1,136,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
10,24.0,638.40,8,6384,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
100,20.0,11777.40,6,176661,3,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1000,17.0,816.25,3,9795,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [42]:
X, X_finaltest, y, y_final_test = train_test_split(df_class,y_out, random_state=314)

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [44]:
def feature_importance(clf, X):
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]
    j = list(X.columns)
    # Print the feature ranking
    print("Feature ranking:")
    l = []
    for f in range(X.shape[1]):
        l.append( [j[indices[f]], importances[indices[f]]])
    return l

def standard_confusion_matrix(y_true, y_pred):
    """Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_true, y_pred)
    return np.array([[tp, fp], [fn, tn]])

def roc_curve(probabilities, labels):
    '''
    INPUT: numpy array, numpy array
    OUTPUT: list, list, list

    Take a numpy array of the predicted probabilities and a numpy array of the
    true labels.
    Return the True Positive Rates, False Positive Rates and Thresholds for the
    ROC curve.
    '''
    
    thresholds = np.sort(probabilities)

    tprs = []
    fprs = []

    num_positive_cases = sum(labels)
    num_negative_cases = len(labels) - num_positive_cases

    for threshold in thresholds:
        # With this threshold, give the prediction of each instance
        predicted_positive = probabilities >= threshold
        # Calculate the number of correctly predicted positive cases
        true_positives = np.sum(predicted_positive * labels)
        # Calculate the number of incorrectly predicted positive cases
        false_positives = np.sum(predicted_positive) - true_positives
        # Calculate the True Positive Rate
        tpr = true_positives / float(num_positive_cases)
        # Calculate the False Positive Rate
        fpr = false_positives / float(num_negative_cases)

        fprs.append(fpr)
        tprs.append(tpr)

    return tprs, fprs, thresholds.tolist()

In [45]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

probabilities = clf.predict_proba(X_test)[:, 1]
tprLR, fprLR, thresholds = roc_curve(probabilities, y_test)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))
preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Train Crossval Accuracy: [ 0.63701068  0.64959929  0.64203028  0.65494212  0.64648264]
Predicted Accuracy: 0.646012998989
Test Accuracy: 0.653846153846
[[1255  715]
 [ 581 1193]]


In [46]:
clf = GradientBoostingClassifier(learning_rate=.01, max_features=20,n_estimators=1000)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

probabilities = clf.predict_proba(X_test)[:, 1]
tprGB, fprGB, thresholds = roc_curve(probabilities, y_test)

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))


feature_importance(clf, X)
    

Train Crossval Accuracy: [ 0.65124555  0.67898486  0.65048976  0.65627783  0.64247551]
Predicted Accuracy: 0.655894702484
Test Accuracy: 0.672008547009
[[1252  644]
 [ 584 1264]]
Feature ranking:


[['AverageWinsToDate', 0.12571117229154566],
 ['PrizeToDate', 0.12000815562109192],
 ['PriorWins', 0.11860810999039119],
 ['currency_KRW', 0.093499916171638608],
 ['currency_EUR', 0.078306581660949337],
 ['Age_at_win', 0.074712793677125022],
 ['SecondWins', 0.065897702406785899],
 ['country_de', 0.038449775102923869],
 ['country_kr', 0.024945819111995775],
 ['currency_Other', 0.024075473674726441],
 ['currency_AUD', 0.020116474713572963],
 ['currency_USD', 0.018452471961945247],
 ['teams_KT Rolster', 0.013805892068191843],
 ['teams_Jin Air Green Wings', 0.012939057971370075],
 ['teams_Team SCV Life', 0.012388655317998494],
 ['teams_Other', 0.011450859415863617],
 ['currency_CNY', 0.010664428029169507],
 ['teams_SK Telecom T1', 0.010115557537946773],
 ['teams_Evil Geniuses', 0.0099878736830856171],
 ['teams_MVP', 0.0099361377430638442],
 ['country_ua', 0.0098536468173657677],
 ['country_fr', 0.0096256847616433879],
 ['teams_Unaffiliated', 0.0091799586316936369],
 ['country_se', 0.007852

In [47]:
clf = RandomForestClassifier(max_depth=10, n_jobs=750, max_features=30)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

probabilities = clf.predict_proba(X_test)[:, 1]
tprRF, fprRF, thresholds = roc_curve(probabilities, y_test)

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

feature_importance(clf, X)

Train Crossval Accuracy: [ 0.6405694   0.66562778  0.65271594  0.66696349  0.64158504]
Predicted Accuracy: 0.653492329582
Test Accuracy: 0.675480769231
[[1213  592]
 [ 623 1316]]
Feature ranking:


[['currency_KRW', 0.18171259896063338],
 ['AverageWinsToDate', 0.14318804044582109],
 ['PrizeToDate', 0.12485092835964931],
 ['currency_EUR', 0.12335003886112623],
 ['PriorWins', 0.09422988116895839],
 ['SecondWins', 0.062606540033975561],
 ['Age_at_win', 0.061951696245694007],
 ['country_de', 0.034108680560540991],
 ['country_kr', 0.018970174779637493],
 ['currency_USD', 0.017386017145826691],
 ['currency_Other', 0.017180026423738625],
 ['teams_Unaffiliated', 0.010273422898161496],
 ['teams_Other', 0.0099989882918741519],
 ['teams_KT Rolster', 0.0091394068797984399],
 ['teams_SK Telecom T1', 0.0075751021842374216],
 ['country_Other', 0.006591864345309094],
 ['country_cn', 0.004995666914307563],
 ['currency_AUD', 0.0046589086537232819],
 ['country_us', 0.0044561346226842562],
 ['teams_MVP', 0.0044283448680854254],
 ['country_fr', 0.0044096138663035876],
 ['currency_CNY', 0.0043156509156029466],
 ['teams_StarCraft II (Samsung Galaxy)', 0.0038749703997337064],
 ['teams_ESC Gaming', 0.003

In [48]:

theme = Theme(json={
    'attrs': {
        'Figure': {
            'background_fill_color': '#2F2F2F',
            'border_fill_color': '#2F2F2F',
            'outline_line_color': '#444444'
            },
        'Axis': {
            'axis_line_color': "white",
            'axis_label_text_color': "white",
            'major_label_text_color': "white",
            'major_tick_line_color': "white",
            'minor_tick_line_color': "white",
            'minor_tick_line_color': "white"
            },
        'Grid': {
            'grid_line_dash': [6, 4],
            'grid_line_alpha': .3
            },
        'Circle': {
            'fill_color': 'lightblue',
            'size': 10,
            },
        'Title': {
            'text_color': "white"
            }
        }
    })
curdoc().theme = theme
#output_notebook()
output_file=('test.html')


TOOLS = [HoverTool(tooltips=[('FPR:','@x'),('TPR','@y')])]
pal = Spectral11
p = figure(x_axis_label='False Positive Rate', y_axis_label='True Positive Rate', tools=TOOLS)
xvals = np.arange(0,1.01,.01)
yvals = np.arange(0,1.01,.01)

p.line(fprLR, tprLR, legend = 'Logistic Regression', line_width=3, line_color=pal[0])
p.line(fprGB, tprGB, legend = 'Gradient Boost', line_width=3, line_color=pal[1])
p.line(fprRF, tprRF, legend = 'Random Forest', line_width=3, line_color=pal[2])
p.line(xvals,yvals, legend = 'Random Chance', line_width=3, line_color=pal[3])
p.legend.location = 'bottom_right'
show(p)

In [49]:
print(list(X.columns))

['Age_at_win', 'AverageWinsToDate', 'PriorWins', 'PrizeToDate', 'SecondWins', 'currency_AUD', 'currency_CNY', 'currency_EUR', 'currency_KRW', 'currency_Other', 'currency_USD', 'teams_CJ Entus', 'teams_Dead Pixels', 'teams_ESC Gaming', 'teams_Evil Geniuses', 'teams_FXOpen e-Sports', 'teams_Incredible Miracle', 'teams_Invictus Gaming', 'teams_Jin Air Green Wings', 'teams_KT Rolster', 'teams_MVP', 'teams_Millenium', 'teams_Old Generations', 'teams_Other', 'teams_PSISTORM Gaming', 'teams_Prime', 'teams_ROOT Gaming', 'teams_SK Telecom T1', 'teams_SlayerS', 'teams_StarCraft II (Samsung Galaxy)', 'teams_StarTale', 'teams_Team Acer', 'teams_Team Empire', 'teams_Team Liquid', 'teams_Team SCV Life', 'teams_Unaffiliated', 'teams_mYinsanity', 'teams_mousesports', 'country_Other', 'country_cn', 'country_de', 'country_fr', 'country_kr', 'country_pl', 'country_ru', 'country_se', 'country_ua', 'country_us']


In [50]:
print(feature_importance(clf,X))

Feature ranking:
[['currency_KRW', 0.18171259896063338], ['AverageWinsToDate', 0.14318804044582109], ['PrizeToDate', 0.12485092835964931], ['currency_EUR', 0.12335003886112623], ['PriorWins', 0.09422988116895839], ['SecondWins', 0.062606540033975561], ['Age_at_win', 0.061951696245694007], ['country_de', 0.034108680560540991], ['country_kr', 0.018970174779637493], ['currency_USD', 0.017386017145826691], ['currency_Other', 0.017180026423738625], ['teams_Unaffiliated', 0.010273422898161496], ['teams_Other', 0.0099989882918741519], ['teams_KT Rolster', 0.0091394068797984399], ['teams_SK Telecom T1', 0.0075751021842374216], ['country_Other', 0.006591864345309094], ['country_cn', 0.004995666914307563], ['currency_AUD', 0.0046589086537232819], ['country_us', 0.0044561346226842562], ['teams_MVP', 0.0044283448680854254], ['country_fr', 0.0044096138663035876], ['currency_CNY', 0.0043156509156029466], ['teams_StarCraft II (Samsung Galaxy)', 0.0038749703997337064], ['teams_ESC Gaming', 0.003136779